In [6]:
import boto3
import json
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

def load_jsonl_from_s3(bucket_name, file_key):
    """
    Load a JSONL file from an S3 bucket using credentials from environment variables.

    Parameters:
    - bucket_name: str - Name of the S3 bucket.
    - file_key: str - Key (path) of the JSONL file in the bucket.

    Returns:
    - List of Python dictionaries loaded from the JSONL file.
    """
    # Get AWS credentials from environment variables
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    region_name = os.getenv("AWS_REGION")
    
    # Initialize an S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )
    
    # Retrieve the file content
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    
    # Parse the JSONL content
    data = [json.loads(line) for line in content.splitlines() if line.strip()]
    
    return data

# Example usage
if __name__ == "__main__":
    bucket_name = "small-reviews584"
    file_key = "data/reviews_small.jsonl"
    
    data = load_jsonl_from_s3(bucket_name, file_key)

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Create a Spark session
spark = SparkSession.builder \
    .appName("AmazonReviewsLocalLR") \
    .master("local[*]") \
    .getOrCreate()

# Convert Python list of dicts to Spark DataFrame
df = spark.createDataFrame(data)

# Ensure the expected columns exist: 'text' for review text and 'rating' for the star rating
# If not present, adjust column names accordingly.
required_columns = ["text", "rating"]
for col_name in required_columns:
    if col_name not in df.columns:
        raise ValueError(f"Expected column '{col_name}' not found in data")

# Create binary label: label=1 if rating >=3 else 0
df = df.withColumn("label", when(col("rating") >= 3, 1).otherwise(0))

# Filter out rows without text
df = df.filter(col("text").isNotNull())

# Define text processing and feature extraction pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=20, regParam=0.001)

# Create a pipeline for convenience
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, lr])

# Split into train and test sets
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Fit the model
model = pipeline.fit(train_df)

# Make predictions
predictions = model.transform(test_df)

# Evaluate model performance
binary_evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = binary_evaluator.evaluate(predictions)

multi_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = multi_evaluator.evaluate(predictions)

print(f"Test AUC: {auc}")
print(f"Test Accuracy: {accuracy}")

# Stop the Spark session
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 15:02:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/11 15:03:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Test AUC: 0.734489855924728
Test Accuracy: 0.8333333333333334
